In [1]:
import pandas as pd
import numpy as np
import sortedcontainers as sc
import pickle
import glob
import random
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.svm import LinearSVC
from scipy.spatial.distance import cdist

In [7]:
arranged_datas = {}

for file in glob.iglob('.\\*'):
    print(file)
    if '.dat' in file:
        try:
            with open(file, 'rb') as f:
                player, games = pickle.load(f)
                arranged_datas[player] = games
        except MemoryError:
            print('error with file : ' + str(file))

.\AxHeart.dat
error with file : .\AxHeart.dat
.\Bbyong.dat
error with file : .\Bbyong.dat
.\Bomber.dat


KeyboardInterrupt: 

In [6]:
arranged_datas.keys()

dict_keys(['/AxHeart/'])

In [3]:
def player_to_number(player):
    return list(arranged_datas.keys()).index(player)

In [4]:
def to_number_array(array2d):
    array = []
    for a in array2d:
        line = []
        if 'NaN' not in a:
            try:
                for number in a:
                    line.append(int(number))
                line[-1] /= 3000.
                array.append(line)
            except:
                pass
        else:
            break
    output = []
    for a, b, c, d, e in zip(array, array[1:], array[2:], array[3:], array[4:]):
        line = a + b + c + d + e
        output.append(line)
    return output

## Creation of a validation set and train set

In [8]:
validation_set = dict.fromkeys(arranged_datas.keys())
train_set = dict.fromkeys(arranged_datas.keys())

In [9]:
def add_game(used_set, player, game):
    if used_set[player] is None:
        used_set[player] = []
    used_set[player].append(game)

In [10]:
for player, games in arranged_datas.items():
    for game in games:
        rdint = random.randint(0, 9)
        if rdint == 9:
            add_game(validation_set, player, game)
        else:
            add_game(train_set, player, game)

In [9]:
len(validation_set['/TYTY/'])

9

In [10]:
len(train_set['/TYTY/'])

38

In [11]:
for game in train_set['/YoDa/']:
    print(game)

[array(['1', '-1', '1', '-1', '-1', '-1', '-1', '16'],
      dtype='<U11'), array(['1', '-1', '1', '-1', '-1', '-1', '-1', '18'],
      dtype='<U11'), array(['1', '-1', '1', '-1', '-1', '-1', '-1', '21'],
      dtype='<U11'), array(['1', '-1', '1', '-1', '-1', '-1', '-1', '22'],
      dtype='<U11'), array(['1', '-1', '1', '-1', '-1', '-1', '-1', '32'],
      dtype='<U11'), array(['1', '-1', '-1', '-1', '-1', '6', '0', '33'],
      dtype='<U11'), array(['1', '-1', '1', '-1', '-1', '-1', '-1', '39'],
      dtype='<U11'), array(['1', '-1', '1', '-1', '-1', '-1', '-1', '45'],
      dtype='<U11'), array(['1', '-1', '-1', '-1', '-1', '8', '0', '49'],
      dtype='<U11'), array(['1', '-1', '1', '-1', '-1', '-1', '-1', '79'],
      dtype='<U11'), array(['1', '-1', '1', '-1', '-1', '-1', '-1', '92'],
      dtype='<U11'), array(['1', '-1', '-1', '-1', '-1', '6', '2', '102'],
      dtype='<U11'), array(['1', '-1', '1', '-1', '-1', '-1', '-1', '106'],
      dtype='<U11'), array(['1', '-1', '1', '-

[array(['1', '-1', '1', '-1', '-1', '-1', '-1', '16'],
      dtype='<U11'), array(['1', '-1', '1', '-1', '-1', '-1', '-1', '21'],
      dtype='<U11'), array(['1', '-1', '1', '-1', '-1', '-1', '-1', '34'],
      dtype='<U11'), array(['1', '-1', '1', '-1', '-1', '-1', '-1', '45'],
      dtype='<U11'), array(['1', '-1', '-1', '-1', '-1', '1', '0', '51'],
      dtype='<U11'), array(['1', '-1', '-1', '-1', '-1', '3', '0', '51'],
      dtype='<U11'), array(['1', '-1', '-1', '-1', '-1', '2', '0', '53'],
      dtype='<U11'), array(['1', '-1', '-1', '-1', '-1', '5', '0', '60'],
      dtype='<U11'), array(['1', '-1', '-1', '-1', '-1', '4', '0', '65'],
      dtype='<U11'), array(['1', '-1', '-1', '-1', '-1', '6', '0', '71'],
      dtype='<U11'), array(['1', '-1', '-1', '-1', '-1', '7', '0', '80'],
      dtype='<U11'), array(['1', '-1', '-1', '-1', '-1', '9', '0', '80'],
      dtype='<U11'), array(['1', '-1', '-1', '-1', '-1', '8', '0', '83'],
      dtype='<U11'), array(['1', '-1', '-1', '-1', '-1

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Test KNN

In [20]:
classifier = KNN(7)

In [21]:
XS = []
YS = []
for player, games in train_set.items():
    xs = to_number_array(np.reshape(games, (-1, 8)))
    ys = []
    for _ in xs:
        ys.append([player_to_number(player)])
    XS.extend(xs)
    YS.extend(ys)
    
classifier.fit(XS, YS)

c:\users\jliermann\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  # This is added back by InteractiveShellApp.init_path()


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=7, p=2,
           weights='uniform')

In [22]:
for player, games in validation_set.items():
    xs = to_number_array(np.reshape(games, (-1, 8)))
    ys = []
    for _ in xs:
        ys.append([player_to_number(player)])
    predictions = classifier.predict(xs)
    print(player_to_number(player))
    pred = {}
    for point in predictions:
        if point not in pred.keys():
            pred[point] = 0
        pred[point] += 1
    if player_to_number(player) == list(pred.keys())[np.argmax(list(pred.values()))]:
        print('yeaaaaaaah')
    else:
        print('')
        print(np.argmax(list(pred.values())))
    print('')
    print('')

0

8


1

6


2

1


3
yeaaaaaaah


4

1


5

14


6

6


7

7


8
yeaaaaaaah


9

13


10
yeaaaaaaah


11

7


12
yeaaaaaaah


13

8


14

5


15

5


16

5


17

6


18
yeaaaaaaah


19

5


20
yeaaaaaaah


21

4


22

4


23

0


24

7


25

7


26

11


27

0


28

3


29
yeaaaaaaah


30

0


31

1


32

4


33

5


34
yeaaaaaaah


35

1


36

3


37
yeaaaaaaah


38

3


39
yeaaaaaaah


40
yeaaaaaaah


41

2


42

6


43

2


44

8


45
yeaaaaaaah


46

4


47

3


48

0


49

1


50

8


51
yeaaaaaaah


52

6


53

0


54

8


55

0


56

6


57
yeaaaaaaah


58

3


59

9


60

1


61
yeaaaaaaah


62

12


63

12


64

4


65
yeaaaaaaah


66

5


67

6


68

17


69

3




In [23]:
pred = {}
for point in predictions:
    if point not in pred.keys():
        pred[point] = 0
    pred[point] += 1

In [24]:
np.argmax(list(pred.values()))

3

In [25]:
pred

{0: 2,
 5: 1,
 6: 1,
 10: 162,
 11: 1,
 18: 9,
 19: 9,
 23: 9,
 30: 27,
 33: 1,
 35: 24,
 37: 13,
 38: 1,
 42: 54,
 43: 7,
 44: 1,
 46: 5,
 49: 26,
 52: 8,
 54: 9,
 55: 44,
 56: 865,
 57: 11,
 59: 5,
 65: 1,
 67: 4,
 69: 60}

## Compare as cloud of points

In [70]:
class CloudAsFunction(object):
    
    step = 50
    minimum_useful = 1000
    maximum_useful = 3000
    
    def __init__(self, cloud):
        self._cloud = cloud
        
    def get_value_at(self, index):
        for value_a, value_b in zip(self._cloud, self._cloud[1:]):
            if value_a[-1] <= index and value_b[-1] > index:
                b_minus_a = np.asarray(value_b[:-1]) - np.asarray(value_a[:-1])
                space = np.asarray(value_b[-1]) - np.asarray(value_a[-1])
                return index, ((b_minus_a/space) * (index - np.asarray(value_a[-1])) + np.asarray(value_a[:-1]))
        return self.get_max(), self._cloud[-1][:-1]
    
    def get_max(self):
        return self._cloud[-1][-1]
    
    def custom_distance_with(self, f2):
        maximum_x = min(min(f2.get_max(), self.get_max()), self.maximum_useful)
        distance = 0
        for i in range(self.minimum_useful, maximum_x, self.step):
            distance += cdist(np.expand_dims(np.asarray(self.get_value_at(i)[1]), axis=0),
                              np.expand_dims(np.asarray(f2.get_value_at(i)[1]), axis=0), metric='cosine')[0, 0]
        return distance*self.step / (maximum_x - self.minimum_useful) if maximum_x > self.minimum_useful else 1000000
    
    @classmethod
    def aggregate(cls, list_of_functions):
        array = []
        maximum_x = 100000000
        for func in list_of_functions:
            if maximum_x > func.get_max():
                maximum_x = func.get_max()
        maximum_x = min(maximum_x, cls.maximum_useful)
        for i in range(cls.minimum_useful, maximum_x, cls.step):
            vector = np.asarray(list_of_functions[0].get_value_at(i)[1]).astype('float64')
            for func in list_of_functions:
                vector += np.asarray(func.get_value_at(i)[1])
            vector = [float(a) / float(len(list_of_functions)) for a in vector]
            vector.append(i)
            array.append(vector)
        return CloudAsFunction(array)
    
    @staticmethod
    def to_number_array(array2d):
        array = []
        for a in array2d:
            line = []
            if 'NaN' not in a:
                try:
                    for number in a:
                        line.append(int(number))
                    line[-1]
                    array.append(line)
                except:
                    pass
            else:
                break
        return array

In [71]:
f1 = CloudAsFunction(CloudAsFunction.to_number_array(list(train_set.values())[0][0]))
f2 = CloudAsFunction(CloudAsFunction.to_number_array(list(train_set.values())[1][1]))

In [72]:
f1.custom_distance_with(f2)

0.39177651600682056

In [73]:
functions_train_set = dict.fromkeys(train_set.keys())
functions_validation_set = dict.fromkeys(train_set.keys())

In [74]:
for player, games in arranged_datas.items():
    print(player)
    for game in games:
        rdint = random.randint(0, 9)
        if rdint == 9:
            add_game(functions_validation_set, player, CloudAsFunction(CloudAsFunction.to_number_array(game)))
        else:
            add_game(functions_train_set, player, CloudAsFunction(CloudAsFunction.to_number_array(game)))
            
for player, games in functions_train_set.items():
    print(player)
    try:
        functions_train_set[player] = [CloudAsFunction.aggregate(games)]
    except Exception as e:
        print('cannot aggregate player, doesn\'t do anything then, pass')
        print(e)
        pass
    

/AxHeart/
/Bbyong/
/Bomber/
/Bunny/
/ByuL/
/Classic/
/CMStormPolt/
/Cure/
/Curious/
/Dark/
/Dear/
/Dream/
/DRG/
/FanTaSy/
/FireCake/
/ForGG/
/Golden/
/Happy/
/herO/
/HuK/
/hydra/
/HyuN/
/iaguz/
/iGJim/
/iGXiGua/
/INnoVation/
/JJAKJI/
/Kane/
/Leenock/
/Life/
/Lilbow/
/LiquidBunny/
/LiquidSnute/
/LiquidTaeJa/
/LiquidTLO/
/MaNa/
/Maru/
/MinChul/
/MMA/
/MyuNgSiK/
/MǂForGG/
/Nerchio/
/ParalyzE/
/PartinG/
/Pigbaby/
/Polt/
/Rain/
/Rogue/
/Sen/
/ShoWTimE/
/Solar/
/soO/
/sOs/
/Soulkey/
/Stardust/
/Stats/
/Stork/
/Super/
/Symbol/
/Trap/
/True/
/TYTY/
/viOLet/
/VortiX/
/Welmu/
/WhiteRa/
/YoDa/
/yoeFWSan/
/YongHwa/
/Zest/
/AxHeart/
/Bbyong/
/Bomber/
cannot aggregate player, doesn't do anything then, pass
list index out of range
/Bunny/
/ByuL/
/Classic/
/CMStormPolt/
/Cure/
/Curious/
/Dark/
/Dear/
/Dream/
/DRG/
/FanTaSy/
/FireCake/
/ForGG/
/Golden/
/Happy/
/herO/
/HuK/
/hydra/
/HyuN/
/iaguz/
/iGJim/
/iGXiGua/
/INnoVation/
/JJAKJI/
/Kane/
/Leenock/
/Life/
/Lilbow/
/LiquidBunny/
/LiquidSnute/
/Liquid

In [81]:
distances = dict.fromkeys(train_set.keys(), [])
good_guess = 0
wrong_guess = 0

In [83]:
for player, games in functions_validation_set.items():
    print('For Player {}'.format(player))
    for game in games:
        min_dist = 10000000000
        min_player = ''
        for ref_player, ref_games in functions_train_set.items():
            try:
                cur_dist = game.custom_distance_with(ref_games[0])
                if min_dist > cur_dist:
                    min_dist = cur_dist
                    min_player = ref_player
                    print('new minimum found ! {}'.format(min_dist))
            except:
                pass
        distances[player].append((min_dist, min_player))
        print('player guessed : {}'.format(min_player))
        if player == min_player:
            good_guess += 1
        else:
            wrong_guess += 1
    break

For Player /AxHeart/
compute distance for ref /AxHeart/
new minimum found ! 0.3011297704498038
compute distance for ref /Bbyong/
compute distance for ref /Bomber/
compute distance for ref /Bunny/
compute distance for ref /ByuL/
compute distance for ref /Classic/
compute distance for ref /CMStormPolt/
new minimum found ! 0.2932763096390436
compute distance for ref /Cure/
compute distance for ref /Curious/
compute distance for ref /Dark/
compute distance for ref /Dear/
compute distance for ref /Dream/
compute distance for ref /DRG/
compute distance for ref /FanTaSy/
compute distance for ref /FireCake/
compute distance for ref /ForGG/
compute distance for ref /Golden/
compute distance for ref /Happy/
compute distance for ref /herO/
compute distance for ref /HuK/
compute distance for ref /hydra/
compute distance for ref /HyuN/
compute distance for ref /iaguz/
new minimum found ! 0.2744816524271007
compute distance for ref /iGJim/
compute distance for ref /iGXiGua/
compute distance for ref 

In [78]:
functions_train_set['/iGXiGua/'][0]